In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

%matplotlib inline

# Load Data

In [38]:
df = pd.read_csv('data/Train.csv')

# EDA

In [39]:
def eda(df):
    df = df.copy()
    
    pg = pd.get_dummies(df.ProductGroup).iloc[:,:-1]
    pgc = pg.columns.to_list()
    df[pgc] = pg
    
    states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Unspecified', 'Utah', 'Vermont', 'Virginia', 'Washington', 'Washington DC', 'West Virginia', 'Wisconsin']
    st = pd.get_dummies(df.state)
    stc = st.columns.to_list()
    df[stc] = st
    remaining = list(set(states) - set(stc))
    srows = df.shape[0]
    scols = len(remaining)
    df[remaining] = pd.DataFrame(np.zeros((srows, scols)), columns=remaining)

    df['MachineHoursCurrentMeter'] = df.MachineHoursCurrentMeter.fillna(0)
    
    df['UsageBand'] = df.UsageBand.fillna('Medium')    
    ub = pd.get_dummies(df.UsageBand).iloc[:,:-1]
    ubc = ub.columns.to_list()
    df[ubc] = ub
    
    df = df.set_index('SalesID')
    keep = ['SalePrice', 'YearMade', 'MachineHoursCurrentMeter'] + pgc + states + ubc
    return df[keep]

In [40]:
df = eda(df)
df.head(2)

,SalePrice,YearMade,MachineHoursCurrentMeter,BL,MG,SSL,TEX,TTT,Alabama,Alaska,...,Unspecified,Utah,Vermont,Virginia,Washington,Washington DC,West Virginia,Wisconsin,High,Low
SalesID,,,,,,,,,,,,,,,,,,,,,
1139246,66000,2004,68.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1139248,57000,1996,4640.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Get X and y

In [42]:
y_train = df.pop('SalePrice')
X_train = df

# Scale data (already split into test/train)

In [43]:
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)

/home/chyld/.conda/envs/py37/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/chyld/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  


# Custom Loss Function

In [44]:
def rmsle(actual, predictions):
    log_diff = np.log(predictions+1) - np.log(actual+1)
    return np.sqrt(np.mean(log_diff**2))

In [45]:
score = make_scorer(rmsle, greater_is_better=False)

# Grid Search

In [46]:
params = {'alpha': np.logspace(-4, 5)}
gs = GridSearchCV(Ridge(), param_grid=params, scoring=score, cv=10, n_jobs=5).fit(X_train, y_train)

# Results

In [47]:
gs.best_params_

{'alpha': 100000.0}

In [48]:
rmsle(y_train, gs.predict(X_train))

/home/chyld/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


0.6191733172866748

# Evaluate Test Data

In [49]:
test_dfx = pd.read_csv('data/test.csv')
test_dfx = test_dfx.set_index('SalesID')
test_dfx.head(2)

,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
SalesID,,,,,,,,,,,,,,,,,,,,,
1222837,902859,1376,121,3,1000,0.0,NaN,1/5/2012 0:00,375L,375,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
1222839,1048320,36526,121,3,2006,4412.0,Medium,1/5/2012 0:00,TX300LC2,TX300,...,None or Unspecified,"12' 4""",None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN


In [50]:
test_dfy = pd.read_csv('data/do_not_open/test_soln.csv')
test_dfy = test_dfy.set_index('SalesID')
test_dfy = test_dfy.drop(columns=['Usage'])
test_dfy.head(2)

,SalePrice
SalesID,
1222837,31000.0
1222839,54000.0


In [51]:
test_df = test_dfx.join(test_dfy)
test_df.head(2)

,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,SalePrice
SalesID,,,,,,,,,,,,,,,,,,,,,
1222837,902859,1376,121,3,1000,0.0,NaN,1/5/2012 0:00,375L,375,...,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,31000.0
1222839,1048320,36526,121,3,2006,4412.0,Medium,1/5/2012 0:00,TX300LC2,TX300,...,"12' 4""",None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN,54000.0


In [52]:
test_df = test_df.reset_index()
test_df = eda(test_df)
y_test = test_df.pop('SalePrice')
X_test = test_df
X_test = scalar.transform(X_test)

/home/chyld/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """


In [55]:
rmsle(y_test, gs.predict(X_test))

0.39234521341234235